http://okmij.org/ftp/continuations/

In [9]:
import Control.Monad
import Control.Applicative
import Control.Monad.Trans
import Control.Monad.Trans.Cont -- continuation monad

:type shift
:type reset

shift :: forall a r. ((a -> r) -> Cont r r) -> Cont r a

reset :: forall r r'. Cont r r -> Cont r' r

In [42]:
-- reset (3 + shift (\k -> 5*2)) - 1

t1 :: Cont r Int
t1 = (-) <$> reset ( (+) <$> return 3 
                         <*> shift (\k -> (*) <$> return 5 <*> return 2) )
         <*> return 1

In [43]:
:type t1
:type runCont
:type runCont t1
:type runCont t1 id

runCont t1 id
runCont t1 (\v -> putStr $ "the value is "++show v)

t1 :: forall r. Cont r Int

runCont :: forall r a. Cont r a -> (a -> r) -> r

runCont t1 :: forall r. (Int -> r) -> r

runCont t1 id :: Int

9

the value is 9

In [46]:
t1 = (-) <$> reset ( (+) <$> return 3 
                         <*> shift (\k -> return (5*2)) )
         <*> return 1

In [47]:
-- We can do more simplifications, this time syntactic
infixl 6 -!,+!
infixl 7 *!

(-!) = liftA2 (-)
(+!) = liftA2 (+)
(*!) = liftA2 (*)

:type (+!)

(+!) :: forall (f :: * -> *) c. (Applicative f, Num c) => f c -> f c -> f c

In [50]:
t1' = reset (return 3 +! shift (\k -> return (5*2)) ) -! return 1

runCont t1' id

9

In [56]:
t2 = reset (return 3 +! shift (\k -> return $ k (5*2))) -! return 1

:type t2
:type runCont t2

runCont t2 id

t2 :: forall c r'. Num c => ContT r' Identity c

runCont t2 :: forall a r. Num a => (a -> r) -> r

12

In [60]:
t3 = reset (return 3 +! shift (\k -> return $ k(k (5*2)))) -! return 1

:type t3
:type runCont t3

runCont t3 id

t3 :: forall c r'. Num c => ContT r' Identity c

runCont t3 :: forall a r. Num a => (a -> r) -> r

15

https://medium.com/@jooyunghan/%EC%BD%94%EB%A3%A8%ED%8B%B4-%EC%86%8C%EA%B0%9C-504cecc89407

https://www.cs.indiana.edu/~sabry/papers/yield.pdf

https://medium.com/yld-blog/continuations-coroutines-fibers-effects-e163dda9dedc

In [262]:
{-# LANGUAGE TypeSynonymInstances, DeriveFunctor #-}
-- suspendable computation
data Iterator i o r = Result r
                    | Susp o (i -> Iterator i o r)
                    deriving Functor
type Yield = Iterator

instance (Show r, Show o) => Show (Iterator i o r) where
  show (Result r) = "Result: " ++ show r
  show (Susp o _) = "Susp (" ++show o++ ") <<...>>"

instance Applicative (Yield i o) where
  pure = Result
  ff <*> fs = ff >>= \ f -> fs >>= \ s -> return (f s)

instance Monad (Yield i o) where
  Result v >>= f = f v
  Susp v k >>= f = Susp v (\x -> k x >>= f)

run e = reset (return (Result e))
yield e = shift (\k -> return (Susp e k))

In [263]:
:type run
:type yield

run :: forall r r' i o. r -> Cont r' (Iterator i o r)

yield :: forall o a r. o -> Cont (Iterator a o r) a

In [287]:
{- ???
g1 =  do yield 3
         run 4

{-
loop f m = each (run m)
  where
    each (Susp v k) = undefined -- each (k (f v))
    each (Result r) = return r
-}

:type g1
:type runCont
:type runCont g1
:type runCont g1 id

runCont (run 4) id
runCont g1 id

runCont g1 (\k -> id k)
-}

In [157]:
:hoogle Control.Monad.Trans.Cont

module Control.Monad.Trans.<s0>Cont</s0>
URL: https://hackage.haskell.org/package/transformers/docs/Control-Monad-Trans-Cont.html
Continuation monads.

Delimited continuation operators are taken from Kenichi Asai and Oleg
Kiselyov's tutorial at CW 2011, "Introduction to programming with
shift and reset"
(<a>http://okmij.org/ftp/continuations/#tutorial</a>).

type <s0>Cont</s0> r = ContT r Identity
URL: https://hackage.haskell.org/package/transformers/docs/Control-Monad-Trans-Cont.html#t:Cont
Continuation monad. <tt>Cont r a</tt> is a CPS ("continuation-passing
style") computation that produces an intermediate result of type
<tt>a</tt> within a CPS computation whose final result type is
<tt>r</tt>.

The <tt>return</tt> function simply creates a continuation which
passes the value on.

The <tt>&gt;&gt;=</tt> operator adds the bound function into the
continuation chain.

module Control.Monad.Trans.<s0>Cont</s0>
URL: https://hackage.haskell.org/package/rerebase/docs/Control-Monad-Trans-Cont.html

type <s0>Cont</s0> r = ContsT r Identity Identity
URL: https://hackage.haskell.org/package/adjunctions/docs/Control-Monad-Trans-Conts.html#t:Cont

module Rebase.Control.Monad.Trans.<s0>Cont</s0>
URL: https://hackage.haskell.org/package/rebase/docs/Rebase-Control-Monad-Trans-Cont.html

<s0>cont</s0> :: ((a -> r) -> r) -> Cont r a
URL: https://hackage.haskell.org/package/transformers/docs/Control-Monad-Trans-Cont.html#v:cont
Construct a continuation-passing computation from a function. (The
inverse of <a>runCont</a>)

<s0>cont</s0> :: ((a -> r) -> r) -> Cont r a
URL: https://hackage.haskell.org/package/adjunctions/docs/Control-Monad-Trans-Conts.html#v:cont

newtype <s0>ContT</s0> r m a
URL: https://hackage.haskell.org/package/transformers/docs/Control-Monad-Trans-Cont.html#t:ContT
The continuation monad transformer. Can be used to add continuation
handling to any type constructor: the <a>Monad</a> instance and most
of the operations do not require <tt>m</tt> to be a monad.

<a>ContT</a> is not a functor on the category of monads, and many
operations cannot be lifted through it.

<s0>ContT</s0> :: ((a -> m r) -> m r) -> ContT r m a
URL: https://hackage.haskell.org/package/transformers/docs/Control-Monad-Trans-Cont.html#v:ContT

module Control.Monad.Trans.<s0>Control</s0>
URL: https://hackage.haskell.org/package/monad-control/docs/Control-Monad-Trans-Control.html
This module defines the type class <a>MonadBaseControl</a>, a subset
of <a>MonadBase</a> into which generic control operations such as
<tt>catch</tt> can be lifted from <tt>IO</tt> or any other base monad.
Instances are based on monad transformers in <a>MonadTransControl</a>,
which includes all standard monad transformers in the
<tt>transformers</tt> library except <tt>ContT</tt>.

See the <a>lifted-base</a> package which uses <tt>monad-control</tt>
to lift <tt>IO</tt> operations from the <tt>base</tt> library (like
<tt>catch</tt> or <tt>bracket</tt>) into any monad that is an instance
of <tt>MonadBase</tt> or <tt>MonadBaseControl</tt>.

See the following tutorial by Michael Snoyman on how to use this
package:

<a>https://www.yesodweb.com/book/monad-control</a>

<h3>Quick implementation guide</h3>

Given a base monad <tt>B</tt> and a stack of transformers <tt>T</tt>:

<ul>
<li>Define instances <tt><a>MonadTransControl</a> T</tt> for all
transformers <tt>T</tt>, using the <tt><a>defaultLiftWith</a></tt> and
<tt><a>defaultRestoreT</a></tt> functions on the constructor and
deconstructor of <tt>T</tt>.</li>
<li>Define an instance <tt><a>MonadBaseControl</a> B B</tt> for the
base monad:<pre>instance MonadBaseControl B B where type StM B a = a
liftBaseWith f = f <a>id</a> restoreM = <a>return</a> </pre></li>
<li>Define instances <tt><a>MonadBaseControl</a> B m =&gt;
<a>MonadBaseControl</a> B (T m)</tt> for all
transformers:<pre>instance MonadBaseControl b m =&gt; MonadBaseControl
b (T m) where type StM (T m) a = <a>ComposeSt</a> T m a liftBaseWith f
= <a>defaultLiftBaseWith</a> restoreM = 